In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score, classification_report, plot_confusion_matrix
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline

from pre_process import num_of_vars, change_vars, make_binary

In [6]:
pd.set_option('display.max_columns',None)

In [7]:
#load in the df
WR = pd.read_pickle('og_women_recode.pkl')
WR.head()

,caseid,v000,v001,v002,v003,v004,v005,v006,v007,v008,v009,v010,v011,v012,v013,v014,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131,v132,v133,v134,v135,v136,v137,v138,bidx_01,bord_01,b0_01,b1_01,b2_01,b3_01,b4_01,b5_01,b6_01,b7_01,b8_01,b9_01,b10_01,b11_01,b12_01,bidx_02,bord_02,b0_02,b1_02,b2_02,b3_02,b4_02,b5_02,b6_02,b7_02,b8_02,b9_02,b10_02,b11_02,b12_02,bidx_03,bord_03,b0_03,b1_03,b2_03,b3_03,b4_03,b5_03,b6_03,b7_03,b8_03,b9_03,b10_03,b11_03,b12_03,bidx_04,bord_04,b0_04,b1_04,b2_04,b3_04,b4_04,b5_04,b6_04,b7_04,b8_04,b9_04,b10_04,b11_04,b12_04,bidx_05,bord_05,b0_05,b1_05,b2_05,b3_05,b4_05,b5_05,b6_05,b7_05,b8_05,b9_05,b10_05,b11_05,b12_05,bidx_06,bord_06,b0_06,b1_06,b2_06,b3_06,b4_06,b5_06,b6_06,b7_06,b8_06,b9_06,b10_06,b11_06,b12_06,bidx_07,bord_07,b0_07,b1_07,b2_07,b3_07,b4_07,b5_07,b6_07,b7_07,b8_07,b9_07,b10_07,b11_07,b12_07,bidx_08,bord_08,b0_08,b1_08,b2_08,b3_08,b4_08,b5_08,b6_08,b7_08,b8_08,b9_08,b10_08,b11_08,b12_08,bidx_09,bord_09,b0_09,b1_09,b2_09,b3_09,b4_09,b5_09,b6_09,b7_09,b8_09,b9_09,b10_09,b11_09,b12_09,bidx_10,bord_10,b0_10,b1_10,b2_10,b3_10,b4_10,b5_10,b6_10,b7_10,b8_10,b9_10,b10_10,b11_10,b12_10,bidx_11,bord_11,b0_11,b1_11,b2_11,b3_11,b4_11,b5_11,b6_11,b7_11,b8_11,b9_11,b10_11,b11_11,b12_11,bidx_12,bord_12,b0_12,b1_12,b2_12,b3_12,b4_12,b5_12,b6_12,b7_12,b8_12,b9_12,b10_12,b11_12,b12_12,bidx_13,bord_13,b0_13,b1_13,b2_13,b3_13,b4_13,b5_13,b6_13,b7_13,b8_13,b9_13,b10_13,b11_13,b12_13,bidx_14,bord_14,b0_14,b1_14,b2_14,b3_14,b4_14,b5_14,b6_14,b7_14,b8_14,b9_14,b10_14,b11_14,b12_14,bidx_15,bord_15,b0_15,b1_15,b2_15,b3_15,b4_15,b5_15,b6_15,b7_15,b8_15,b9_15,b10_15,b11_15,b12_15,bidx_16,bord_16,b0_16,b1_16,b2_16,b3_16,b4_16,b5_16,b6_16,b7_16,b8_16,b9_16,b10_16,b11_16,b12_16,bidx_17,bord_17,b0_17,b1_17,b2_17,b3_17,b4_17,b5_17,b6_17,b7_17,b8_17,b9_17,b10_17,b11_17,b12_17,bidx_18,bord_18,b0_18,b1_18,b2_18,b3_18,b4_18,b5_18,b6_18,b7_18,b8_18,b9_18,b10_18,b11_18,b12_18,bidx_19,bord_19,b0_19,b1_19,b2_19,b3_19,b4_19,b5_19,b6_19,b7_19,b8_19,b9_19,b10_19,b11_19,b12_19,bidx_20,bord_20,b0_20,b1_20,b2_20,b3_20,b4_20,b5_20,b6_20,b7_20,b8_20,b9_20,b10_20,b11_20,b12_20,v201,v202,v203,v204,v205,v206,v207,v208,v209,v210,v211,v212,v213,v214,v215,v216,v217,v218,v219,v220,v221,v222,v223,v224,v301,v302,v303,v304_01,v305_01,v306_01,v307_01,v308_01,v309_01,v304_02,v305_02,v306_02,v307_02,v308_02,v309_02,v304_03,v305_03,v306_03,v307_03,v308_03,v309_03,v304_04,v305_04,v306_04,v307_04,v308_04,v309_04,v304_05,v305_05,v306_05,v307_05,v308_05,v309_05,v304_06,v305_06,v306_06,v307_06,v308_06,v309_06,v304_07,v305_07,v306_07,v307_07,v308_07,v309_07,v304_08,v305_08,v306_08,v307_08,v308_08,v309_08,v304_09,v305_09,v306_09,v307_09,v308_09,v309_09,v304_10,v305_10,v306_10,v307_10,v308_10,v309_10,v304_11,v305_11,v306_11,v307_11,v308_11,v309_11,v304_12,v305_12,v306_12,v307_12,v308_12,v309_12,v304_13,v305_13,v306_13,v307_13,v308_13,v309_13,v304_14,v305_14,v306_14,v307_14,v308_14,v309_14,v304_15,v305_15,v306_15,v307_15,v308_15,v309_15,v310,v311,v312,v313,v314,v315,v316,v317,v318,v319,v320,v321,v322,v323,v324,v325,v326,v327,v328,v329,v330,v331,v332,v333,v334,v335,v336,v337,v338,v339,v340,v341,v342,v343,v344,v345,v346,v347,v348,v349,v350,v351,v352,v353,v354,v355,v356,v357,v358,v359,v360,v361,v362,v363,v364,v365,v366,v367,v368,cpidx_1,cp1_1,cp2_1,cp3_1,cp4_1,cp5_1,cp6_1,cp7_1,cp8_1,cp9_1,cpidx_2,cp1_2,cp2_2,cp3_2,cp4_2,cp5_2,cp6_2,cp7_2,cp8_2,cp9_2,cpidx_3,cp1_3,cp2_3,cp3_3,cp4_3,cp5_3,cp6_3,cp7_3,cp8_3,cp9_3,cpidx_4,cp1_4,cp2_4,cp3_4,cp4_4,cp5_4,cp6_4,cp7_4,cp8_4,cp9_4,cpidx_5,cp1_5,cp2_5,cp3_5,cp4_5,cp5_5,cp6_5,cp7_5,cp8_5,cp9_5,cpidx_6,cp1_6,cp2_6,cp3_6,cp4_6,cp5_6,cp6_6,cp7_6,cp8_6,cp9_6,cpidx_7,cp1_7,cp2_7,cp3_7,cp4_7,cp5_7,cp6_7,cp7_7,cp8_7,cp9_7,midx_1,m1_1,m2_1,m3_1,m4_1,m5_1,m6_1,m7_1,m8_1,m9_1,midx_2,m1_2,m2_2,m3_2,m4_2,m5_2,m6_2,m7_2,m8_2,m9_2,midx_3,m1_3,m2_3,m3_3,m4_3,m5_3,m6_3,m7_3,m8_3,m9_3,midx_4,m1_4,m2_4,m3_4,m4_4,m5_4,m6_4,m7_4,m8_4,

## Second model

For the second model, I will filter the data down to:

- (rows st v213 = 0) women that are not currently pregnant or do not think that they are pregnant at the time of the survey. 
- women that are sexually active with men 

I will look at the columns/features: 

- caseid
- (v012) current age 
- (v106) highest year of education 
- (v108) literacy 
- (v130) Religion 
- (v201) total number of children born 
- (v301) knowledge of any method 
- (v302) ever use of any method 
- (v303) knows source for modern method 
- and more...


The target is column v313, current use by method type.

(columns starting with b9 are ones that consider if they have a child living with them, perhaps include that in the next iteration)

I will start by just grabbing the columns i find impt

### filter just the ladies who are not pregnant, or don't know that they're pregnant

In [8]:
non_preg = WR[WR['v213'] == 'no or unsure']

In [9]:
non_preg.head()

,caseid,v000,v001,v002,v003,v004,v005,v006,v007,v008,v009,v010,v011,v012,v013,v014,v101,v102,v103,v104,v105,v106,v107,v108,v109,v110,v111,v112,v113,v114,v115,v116,v117,v118,v119,v120,v121,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131,v132,v133,v134,v135,v136,v137,v138,bidx_01,bord_01,b0_01,b1_01,b2_01,b3_01,b4_01,b5_01,b6_01,b7_01,b8_01,b9_01,b10_01,b11_01,b12_01,bidx_02,bord_02,b0_02,b1_02,b2_02,b3_02,b4_02,b5_02,b6_02,b7_02,b8_02,b9_02,b10_02,b11_02,b12_02,bidx_03,bord_03,b0_03,b1_03,b2_03,b3_03,b4_03,b5_03,b6_03,b7_03,b8_03,b9_03,b10_03,b11_03,b12_03,bidx_04,bord_04,b0_04,b1_04,b2_04,b3_04,b4_04,b5_04,b6_04,b7_04,b8_04,b9_04,b10_04,b11_04,b12_04,bidx_05,bord_05,b0_05,b1_05,b2_05,b3_05,b4_05,b5_05,b6_05,b7_05,b8_05,b9_05,b10_05,b11_05,b12_05,bidx_06,bord_06,b0_06,b1_06,b2_06,b3_06,b4_06,b5_06,b6_06,b7_06,b8_06,b9_06,b10_06,b11_06,b12_06,bidx_07,bord_07,b0_07,b1_07,b2_07,b3_07,b4_07,b5_07,b6_07,b7_07,b8_07,b9_07,b10_07,b11_07,b12_07,bidx_08,bord_08,b0_08,b1_08,b2_08,b3_08,b4_08,b5_08,b6_08,b7_08,b8_08,b9_08,b10_08,b11_08,b12_08,bidx_09,bord_09,b0_09,b1_09,b2_09,b3_09,b4_09,b5_09,b6_09,b7_09,b8_09,b9_09,b10_09,b11_09,b12_09,bidx_10,bord_10,b0_10,b1_10,b2_10,b3_10,b4_10,b5_10,b6_10,b7_10,b8_10,b9_10,b10_10,b11_10,b12_10,bidx_11,bord_11,b0_11,b1_11,b2_11,b3_11,b4_11,b5_11,b6_11,b7_11,b8_11,b9_11,b10_11,b11_11,b12_11,bidx_12,bord_12,b0_12,b1_12,b2_12,b3_12,b4_12,b5_12,b6_12,b7_12,b8_12,b9_12,b10_12,b11_12,b12_12,bidx_13,bord_13,b0_13,b1_13,b2_13,b3_13,b4_13,b5_13,b6_13,b7_13,b8_13,b9_13,b10_13,b11_13,b12_13,bidx_14,bord_14,b0_14,b1_14,b2_14,b3_14,b4_14,b5_14,b6_14,b7_14,b8_14,b9_14,b10_14,b11_14,b12_14,bidx_15,bord_15,b0_15,b1_15,b2_15,b3_15,b4_15,b5_15,b6_15,b7_15,b8_15,b9_15,b10_15,b11_15,b12_15,bidx_16,bord_16,b0_16,b1_16,b2_16,b3_16,b4_16,b5_16,b6_16,b7_16,b8_16,b9_16,b10_16,b11_16,b12_16,bidx_17,bord_17,b0_17,b1_17,b2_17,b3_17,b4_17,b5_17,b6_17,b7_17,b8_17,b9_17,b10_17,b11_17,b12_17,bidx_18,bord_18,b0_18,b1_18,b2_18,b3_18,b4_18,b5_18,b6_18,b7_18,b8_18,b9_18,b10_18,b11_18,b12_18,bidx_19,bord_19,b0_19,b1_19,b2_19,b3_19,b4_19,b5_19,b6_19,b7_19,b8_19,b9_19,b10_19,b11_19,b12_19,bidx_20,bord_20,b0_20,b1_20,b2_20,b3_20,b4_20,b5_20,b6_20,b7_20,b8_20,b9_20,b10_20,b11_20,b12_20,v201,v202,v203,v204,v205,v206,v207,v208,v209,v210,v211,v212,v213,v214,v215,v216,v217,v218,v219,v220,v221,v222,v223,v224,v301,v302,v303,v304_01,v305_01,v306_01,v307_01,v308_01,v309_01,v304_02,v305_02,v306_02,v307_02,v308_02,v309_02,v304_03,v305_03,v306_03,v307_03,v308_03,v309_03,v304_04,v305_04,v306_04,v307_04,v308_04,v309_04,v304_05,v305_05,v306_05,v307_05,v308_05,v309_05,v304_06,v305_06,v306_06,v307_06,v308_06,v309_06,v304_07,v305_07,v306_07,v307_07,v308_07,v309_07,v304_08,v305_08,v306_08,v307_08,v308_08,v309_08,v304_09,v305_09,v306_09,v307_09,v308_09,v309_09,v304_10,v305_10,v306_10,v307_10,v308_10,v309_10,v304_11,v305_11,v306_11,v307_11,v308_11,v309_11,v304_12,v305_12,v306_12,v307_12,v308_12,v309_12,v304_13,v305_13,v306_13,v307_13,v308_13,v309_13,v304_14,v305_14,v306_14,v307_14,v308_14,v309_14,v304_15,v305_15,v306_15,v307_15,v308_15,v309_15,v310,v311,v312,v313,v314,v315,v316,v317,v318,v319,v320,v321,v322,v323,v324,v325,v326,v327,v328,v329,v330,v331,v332,v333,v334,v335,v336,v337,v338,v339,v340,v341,v342,v343,v344,v345,v346,v347,v348,v349,v350,v351,v352,v353,v354,v355,v356,v357,v358,v359,v360,v361,v362,v363,v364,v365,v366,v367,v368,cpidx_1,cp1_1,cp2_1,cp3_1,cp4_1,cp5_1,cp6_1,cp7_1,cp8_1,cp9_1,cpidx_2,cp1_2,cp2_2,cp3_2,cp4_2,cp5_2,cp6_2,cp7_2,cp8_2,cp9_2,cpidx_3,cp1_3,cp2_3,cp3_3,cp4_3,cp5_3,cp6_3,cp7_3,cp8_3,cp9_3,cpidx_4,cp1_4,cp2_4,cp3_4,cp4_4,cp5_4,cp6_4,cp7_4,cp8_4,cp9_4,cpidx_5,cp1_5,cp2_5,cp3_5,cp4_5,cp5_5,cp6_5,cp7_5,cp8_5,cp9_5,cpidx_6,cp1_6,cp2_6,cp3_6,cp4_6,cp5_6,cp6_6,cp7_6,cp8_6,cp9_6,cpidx_7,cp1_7,cp2_7,cp3_7,cp4_7,cp5_7,cp6_7,cp7_7,cp8_7,cp9_7,midx_1,m1_1,m2_1,m3_1,m4_1,m5_1,m6_1,m7_1,m8_1,m9_1,midx_2,m1_2,m2_2,m3_2,m4_2,m5_2,m6_2,m7_2,m8_2,m9_2,midx_3,m1_3,m2_3,m3_3,m4_3,m5_3,m6_3,m7_3,m8_3,m9_3,midx_4,m1_4,m2_4,m3_4,m4_4,m5_4,m6_4,m7_4,m8_4,

### now just grab the columns I think are impt for now

In [10]:
non_preg = non_preg[['caseid', 'v012', 'v106', 'v108', 'v109', 
                     'v120', 'v121', 'v123', 'v124','v125','v126',
                     'v130', 'v137', 'v201', 'v217', 'v218', 'v301',
                     'v302', 'v303', 'v406', 'v511', 'v523', 'v525',
                     'v526', 'v605', 'v606', 'v607', 'v610', 'v612',
                     'v701', 'v703', 'v705', 'v711', 'v712', 'v714', 
                     's110', 's315g', 's315j', 's317a', 's342',
                     's703', 'v313']]
non_preg.head()


,caseid,v012,v106,v108,v109,v120,v121,v123,v124,v125,v126,v130,v137,v201,v217,v218,v301,v302,v303,v406,v511,v523,v525,v526,v605,v606,v607,v610,v612,v701,v703,v705,v711,v712,v714,s110,s315g,s315j,s317a,s342,s703,v313
0,11 1 1 2,41,secondary,reads easily,no,yes,no,yes,no,NaN,NaN,muslim,0,6,after period ended,5,knows modern method,used modern method,knows sourc.mod.meth,no,15,NaN,15,4,wants no more,NaN,fatalistic,NaN,NaN,primary,reads easily,agriculture,yes,NaN,yes,junior high,NaN,NaN,NaN,no,primary,no method
1,11 1 4 4,27,secondary,reads easily,yes,yes,yes,yes,no,NaN,NaN,muslim,4,5,after period ended,5,knows modern method,never used,knows sourc.mod.meth,yes,18,NaN,18,no intercourse,"wants, unsure timing",NaN,husband disapproves,NaN,NaN,secondary,reads easily,"prof, tech, cleric.",no,NaN,no,senior high,NaN,NaN,NaN,no,senior high,no method
2,11 1 6 1,43,secondary,reads easily,yes,no,no,no,no,NaN,NaN,muslim,0,8,middle of the cycle,8,knows modern method,used modern method,knows sourc.mod.meth,no,15,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,secondary,reads easily,"sales, services",no,NaN,no,junior high,NaN,NaN,NaN,no,junior high,no method
3,11 1 8 2,41,primary,reads easily,no,yes,no,yes,no,NaN,NaN,muslim,2,9,after period ended,9,knows modern method,never used,knows sourc.mod.meth,no,18,NaN,18,3,"wants, unsure timing",NaN,husband disapproves,NaN,NaN,secondary,reads easily,"prof, tech, cleric.",yes,NaN,yes,primary school,NaN,NaN,NaN,no,senior high,no method
4,11 1 9 2,29,secondary,reads easily,yes,yes,no,yes,no,NaN,NaN,muslim,1,3,after period ended,2,knows modern method,never used,knows sourc.mod.meth,no,17,NaN,17,5,"wants, unsure timing",NaN,other,NaN,NaN,secondary,reads easily,agriculture,no,NaN,no,junior high,NaN,NaN,NaN,no,senior high,no method


In [11]:
non_preg.shape

(11132, 42)

In [12]:
#rename the columns
non_preg.rename(columns = {'v012':'Age', 'v106':'Education_level', 
                           'v108':'Literacy','v130':'Religion',
                           'v109':'Reads_newspaper_once_per_week',
                           'v120':'Has_radio',
                           'v121':'Has_TV',
                           'v123':'Has_transportation1',
                           'v124':'Has_transportation2',
                           'v125':'Has_transportation3',
                           'v126':'Has_transportation4',
                           'v137':'Num_children_in_residence',
                           'v201':'Num_children_born',
                           'v217':'Knowledge_of_ovulatory_cycle', 
                           'v218':'Num_living_children',
                           'v301':'Knowledge_of_methods', 
                           'v302':'Used_any_method',
                           'v303':'Knows_method_source', 
                           'v406':'currently_abstaining',
                           'v511':'age_at_first_marriage',
                           'v523':'currently_living_w_parents',
                           'v525':'age_at_first_intercourse',
                           'v526':'freq_of_intercourse',
                           'v605':'desire_for_more_kids',
                           'v606':'attitude_at_becoming_pregnant',
                           'v607':'main_reason_for_non_use',
                           'v610':'husband_approves_FP',
                           'v612':'discussed_FP_w_partner',
                           'v701':'Husbands_edu_level',
                           'v703':'Husbands_literacy',
                           'v705':'Husbands_occupation',
                           'v711':'Woman_worked_after_marriage',
                           'v712':'Woman_worked_never_married',
                           'v714':'Woman_currently_working',
                           's110':'highest_level_of_edu',
                           's315g':'why_not_taking_pill',
                           's315j':'why_not_getting_injection',
                           's317a':'cost_for_method_in_rupiah',
                           's342': 'heard_FP_message_in_last_month',
                           's703':'hubby_highest_level_edu',
                           'v313':'Current_method'}, inplace = True)
non_preg.head()


,caseid,Age,Education_level,Literacy,Reads_newspaper_once_per_week,Has_radio,Has_TV,Has_transportation1,Has_transportation2,Has_transportation3,Has_transportation4,Religion,Num_children_in_residence,Num_children_born,Knowledge_of_ovulatory_cycle,Num_living_children,Knowledge_of_methods,Used_any_method,Knows_method_source,currently_abstaining,age_at_first_marriage,currently_living_w_parents,age_at_first_intercourse,freq_of_intercourse,desire_for_more_kids,attitude_at_becoming_pregnant,main_reason_for_non_use,husband_approves_FP,discussed_FP_w_partner,Husbands_edu_level,Husbands_literacy,Husbands_occupation,Woman_worked_after_marriage,Woman_worked_never_married,Woman_currently_working,highest_level_of_edu,why_not_taking_pill,why_not_getting_injection,cost_for_method_in_rupiah,heard_FP_message_in_last_month,hubby_highest_level_edu,Current_method
0,11 1 1 2,41,secondary,reads easily,no,yes,no,yes,no,NaN,NaN,muslim,0,6,after period ended,5,knows modern method,used modern method,knows sourc.mod.meth,no,15,NaN,15,4,wants no more,NaN,fatalistic,NaN,NaN,primary,reads easily,agriculture,yes,NaN,yes,junior high,NaN,NaN,NaN,no,primary,no method
1,11 1 4 4,27,secondary,reads easily,yes,yes,yes,yes,no,NaN,NaN,muslim,4,5,after period ended,5,knows modern method,never used,knows sourc.mod.meth,yes,18,NaN,18,no intercourse,"wants, unsure timing",NaN,husband disapproves,NaN,NaN,secondary,reads easily,"prof, tech, cleric.",no,NaN,no,senior high,NaN,NaN,NaN,no,senior high,no method
2,11 1 6 1,43,secondary,reads easily,yes,no,no,no,no,NaN,NaN,muslim,0,8,middle of the cycle,8,knows modern method,used modern method,knows sourc.mod.meth,no,15,NaN,15,NaN,NaN,NaN,NaN,NaN,NaN,secondary,reads easily,"sales, services",no,NaN,no,junior high,NaN,NaN,NaN,no,junior high,no method
3,11 1 8 2,41,primary,reads easily,no,yes,no,yes,no,NaN,NaN,muslim,2,9,after period ended,9,knows modern method,never used,knows sourc.mod.meth,no,18,NaN,18,3,"wants, unsure timing",NaN,husband disapproves,NaN,NaN,secondary,reads easily,"prof, tech, cleric.",yes,NaN,yes,primary school,NaN,NaN,NaN,no,senior high,no method
4,11 1 9 2,29,secondary,reads easily,yes,yes,no,yes,no,NaN,NaN,muslim,1,3,after period ended,2,knows modern method,never used,knows sourc.mod.meth,no,17,NaN,17,5,"wants, unsure timing",NaN,other,NaN,NaN,secondary,reads easily,agriculture,no,NaN,no,junior high,NaN,NaN,NaN,no,senior high,no method


In [13]:
#check which columns are just nans
for column in non_preg.columns:
    print(num_of_vars(non_preg, column))

Num of caseid variables: 11132
    17  4 57  2    1
    32 38 28  2    1
    32506 99  2    1
    16504 66  2    1
    34  9 96  1    1
                  ..
    35 41 47  2    1
    51 17 41  2    1
    34  8  5  1    1
    34500 42  2    1
    31514172  2    1
Name: caseid, Length: 11132, dtype: int64 

None
Num of Age variables: 35
25    593
30    555
27    496
35    474
32    471
26    438
45    420
28    418
29    404
22    389
23    388
24    384
31    373
40    373
33    342
47    330
34    328
37    323
42    314
36    307
38    280
43    278
48    274
39    270
20    264
21    258
41    250
44    241
49    226
46    225
19    155
18    148
17     75
16     47
15     21
Name: Age, dtype: int64 

None
Num of Education_level variables: 4
primary         6316
no education    2516
secondary       2068
higher           232
Name: Education_level, dtype: int64 

None
Num of Literacy variables: 3
reads easily            6135
cannot read             3234
reads with difficult    1762
Name

In [14]:
#some of these columns have no values so i'm gonna drop them
non_preg.drop(columns = ['currently_living_w_parents','husband_approves_FP', 'discussed_FP_w_partner', 'Woman_worked_never_married', 'attitude_at_becoming_pregnant', 'why_not_taking_pill', 'why_not_getting_injection', 'cost_for_method_in_rupiah', 'Has_transportation3', 'Has_transportation4'], inplace = True)


In [15]:
non_preg.head()

,caseid,Age,Education_level,Literacy,Reads_newspaper_once_per_week,Has_radio,Has_TV,Has_transportation1,Has_transportation2,Religion,Num_children_in_residence,Num_children_born,Knowledge_of_ovulatory_cycle,Num_living_children,Knowledge_of_methods,Used_any_method,Knows_method_source,currently_abstaining,age_at_first_marriage,age_at_first_intercourse,freq_of_intercourse,desire_for_more_kids,main_reason_for_non_use,Husbands_edu_level,Husbands_literacy,Husbands_occupation,Woman_worked_after_marriage,Woman_currently_working,highest_level_of_edu,heard_FP_message_in_last_month,hubby_highest_level_edu,Current_method
0,11 1 1 2,41,secondary,reads easily,no,yes,no,yes,no,muslim,0,6,after period ended,5,knows modern method,used modern method,knows sourc.mod.meth,no,15,15,4,wants no more,fatalistic,primary,reads easily,agriculture,yes,yes,junior high,no,primary,no method
1,11 1 4 4,27,secondary,reads easily,yes,yes,yes,yes,no,muslim,4,5,after period ended,5,knows modern method,never used,knows sourc.mod.meth,yes,18,18,no intercourse,"wants, unsure timing",husband disapproves,secondary,reads easily,"prof, tech, cleric.",no,no,senior high,no,senior high,no method
2,11 1 6 1,43,secondary,reads easily,yes,no,no,no,no,muslim,0,8,middle of the cycle,8,knows modern method,used modern method,knows sourc.mod.meth,no,15,15,NaN,NaN,NaN,secondary,reads easily,"sales, services",no,no,junior high,no,junior high,no method
3,11 1 8 2,41,primary,reads easily,no,yes,no,yes,no,muslim,2,9,after period ended,9,knows modern method,never used,knows sourc.mod.meth,no,18,18,3,"wants, unsure timing",husband disapproves,secondary,reads easily,"prof, tech, cleric.",yes,yes,primary school,no,senior high,no method
4,11 1 9 2,29,secondary,reads easily,yes,yes,no,yes,no,muslim,1,3,after period ended,2,knows modern method,never used,knows sourc.mod.meth,no,17,17,5,"wants, unsure timing",other,secondary,reads easily,agriculture,no,no,junior high,no,senior high,no method


In [16]:
#now check our vars to see what we got for each column
for column in non_preg.columns:
    print(num_of_vars(non_preg, column))

Num of caseid variables: 11132
    17  4 57  2    1
    32 38 28  2    1
    32506 99  2    1
    16504 66  2    1
    34  9 96  1    1
                  ..
    35 41 47  2    1
    51 17 41  2    1
    34  8  5  1    1
    34500 42  2    1
    31514172  2    1
Name: caseid, Length: 11132, dtype: int64 

None
Num of Age variables: 35
25    593
30    555
27    496
35    474
32    471
26    438
45    420
28    418
29    404
22    389
23    388
24    384
31    373
40    373
33    342
47    330
34    328
37    323
42    314
36    307
38    280
43    278
48    274
39    270
20    264
21    258
41    250
44    241
49    226
46    225
19    155
18    148
17     75
16     47
15     21
Name: Age, dtype: int64 

None
Num of Education_level variables: 4
primary         6316
no education    2516
secondary       2068
higher           232
Name: Education_level, dtype: int64 

None
Num of Literacy variables: 3
reads easily            6135
cannot read             3234
reads with difficult    1762
Name

Will make the following changes:

**Binary changes**
- Religion: Binary (muslim or nah, since most are muslim) 
- Num_of_children_in_residence: Binary (1 if at least 1 kid, 0 otherwise)
- Knowledge_of_ovulatory_cycle: Binary (0 if 'don't know' 1 otherwise)
- Knows_method_source: Binary (y/n)
- Currently_abstaining: Binary (y/n)
- desire_for_more_kids: Binary ( 1 if 'wants within 2 yrs', 0 otherwise)
- Woman_worked_after_marriage: Binary (y/n) #make nans = 0
- Woman_currently_working: Binary (y/n)
- heard_FP_message_in_last_month: Binary (y/n)


**Ordinal Changes**
- Education_level: Ordinal
- Literacy: Ordinal
- Knowledge of methods: Ordinal
- Used_any_method: Ordinal
- Husbands_edu_level: ordinal #make nans = 0
- Husbands_occupation_level: Ordinal ?
- highest_level_of_edu: ordinal
- hubby_highest_level_edu: Ordinal
- current_method: Ordinal


**Value Changes**
- age_at_first_intercourse: replace 'inconsistent' and 'refused to answer' with the median
- frequency_of_intercourse: replace 'no intercourse' with 0 and 'refused to answer' with the median





**Grouping**
- Reads_newspaper_once_per_week, Has_radio, and Has_TV into one ordinal column 
    - "Good_media_exposure" = sum of those binary vars (makes it ordinal)
    
- Has_transportation1 & 2 into one binary column
    - Has_transportation = 1 (if a 1 in either above column, 0 otherwise)
    
Drop the following:
- Reason_for-non_use (data leakage)

In [17]:
non_preg.drop(columns = 'main_reason_for_non_use', inplace = True)

### do some basic EDA on a few features

In [18]:
#checking out if any of the abstaining women are using contraception
mask = non_preg['currently_abstaining'] == 'yes'# & str(non_preg['Current_method']) != 'no method'
non_preg[mask]['Current_method'].value_counts()
#looks like there are 138 women on FP even though they're abstaining
#does it make sense to include them?


no method             511
modern method         109
traditional method     29
Name: Current_method, dtype: int64

In [19]:
#also gonna check the ones that have not had intercourse
intercourse_mask = non_preg['age_at_first_intercourse'] == 'not had intercourse'
non_preg[intercourse_mask]['Current_method'].value_counts()
#looks like only one of them is on contraception
#it doesn't really make sense to include them though


no method             34
modern method          1
traditional method     0
Name: Current_method, dtype: int64

In [20]:
#reset the index before changing the vars
non_preg.reset_index(inplace = True, drop = True)


In [21]:
non_preg.tail(3)

,caseid,Age,Education_level,Literacy,Reads_newspaper_once_per_week,Has_radio,Has_TV,Has_transportation1,Has_transportation2,Religion,Num_children_in_residence,Num_children_born,Knowledge_of_ovulatory_cycle,Num_living_children,Knowledge_of_methods,Used_any_method,Knows_method_source,currently_abstaining,age_at_first_marriage,age_at_first_intercourse,freq_of_intercourse,desire_for_more_kids,Husbands_edu_level,Husbands_literacy,Husbands_occupation,Woman_worked_after_marriage,Woman_currently_working,highest_level_of_edu,heard_FP_message_in_last_month,hubby_highest_level_edu,Current_method
11129,74500 75 2,44,primary,reads easily,no,no,yes,no,yes,muslim,1,8,after period ended,5,knows modern method,used modern method,knows sourc.mod.meth,no,19,19,3,wants no more,primary,reads easily,"sales, services",yes,yes,primary school,yes,primary,modern method
11130,74500 82 2,25,secondary,reads easily,yes,yes,no,no,no,muslim,3,4,middle of the cycle,4,knows modern method,never used,knows sourc.mod.meth,no,17,17,7,wants no more,secondary,reads easily,"sales, services",no,no,junior high,yes,senior high,no method
11131,74500 85 2,22,secondary,reads easily,yes,no,no,no,no,muslim,1,1,after period ended,1,knows modern method,never used,knows sourc.mod.meth,no,21,21,2,wants after 2+ years,higher,reads easily,"prof, tech, cleric.",yes,yes,senior high,no radio or tv,academy,no method


## Change the binary values first

In [23]:
#change the religion
make_binary(non_preg, 'Religion', 'muslim', 1)

#change the numof children in residence
make_binary(non_preg, 'Num_children_in_residence', '0')

#change the knowledge of ovulatory cylce 
make_binary(non_preg, 'Knowledge_of_ovulatory_cycle', "don't know")

#change Knows_method_source
make_binary(non_preg, 'Knows_method_source', "doesn't know source")

#change currently_abstaining
make_binary(non_preg, 'currently_abstaining', 'no')

#change the desire for more children
make_binary(non_preg, 'desire_for_more_kids', 'wants within 2 years', 1)

#change Woman_worked_after_marriage
make_binary(non_preg, 'Woman_worked_after_marriage', 'yes', 1)

#change Woman_currently_working
make_binary(non_preg, 'Woman_currently_working', 'yes', 1)

#change heard_FP_message_in_last_month
make_binary(non_preg, 'heard_FP_message_in_last_month', 'yes')

,caseid,Age,Education_level,Literacy,Reads_newspaper_once_per_week,Has_radio,Has_TV,Has_transportation1,Has_transportation2,Religion,Num_children_in_residence,Num_children_born,Knowledge_of_ovulatory_cycle,Num_living_children,Knowledge_of_methods,Used_any_method,Knows_method_source,currently_abstaining,age_at_first_marriage,age_at_first_intercourse,freq_of_intercourse,desire_for_more_kids,Husbands_edu_level,Husbands_literacy,Husbands_occupation,Woman_worked_after_marriage,Woman_currently_working,highest_level_of_edu,heard_FP_message_in_last_month,hubby_highest_level_edu,Current_method,Religion_dup,Num_children_in_residence_dup,Knowledge_of_ovulatory_cycle_dup,Knows_method_source_dup,currently_abstaining_dup,desire_for_more_kids_dup,Woman_worked_after_marriage_dup,Woman_currently_working_dup,heard_FP_message_in_last_month_dup
0,11 1 1 2,41,secondary,reads easily,no,yes,no,yes,no,muslim,0,6,after period ended,5,knows modern method,used modern method,knows sourc.mod.meth,no,15,15,4,wants no more,primary,reads easily,agriculture,yes,yes,junior high,no,primary,no method,1,0,1,1,0,0,1,1,1
1,11 1 4 4,27,secondary,reads easily,yes,yes,yes,yes,no,muslim,4,5,after period ended,5,knows modern method,never used,knows sourc.mod.meth,yes,18,18,no intercourse,"wants, unsure timing",secondary,reads easily,"prof, tech, cleric.",no,no,senior high,no,senior high,no method,1,1,1,1,1,0,0,0,1
2,11 1 6 1,43,secondary,reads easily,yes,no,no,no,no,muslim,0,8,middle of the cycle,8,knows modern method,used modern method,knows sourc.mod.meth,no,15,15,NaN,NaN,secondary,reads easily,"sales, services",no,no,junior high,no,junior high,no method,1,0,1,1,0,0,0,0,1
3,11 1 8 2,41,primary,reads easily,no,yes,no,yes,no,muslim,2,9,after period ended,9,knows modern method,never used,knows sourc.mod.meth,no,18,18,3,"wants, unsure timing",secondary,reads easily,"prof, tech, cleric.",yes,yes,primary school,no,senior high,no method,1,1,1,1,0,0,1,1,1
4,11 1 9 2,29,secondary,reads easily,yes,yes,no,yes,no,muslim,1,3,after period ended,2,knows modern method,never used,knows sourc.mod.meth,no,17,17,5,"wants, unsure timing",secondary,reads easily,agriculture,no,no,junior high,no,senior high,no method,1,1,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11127,74500 66 2,17,secondary,reads easily,yes,no,no,no,no,muslim,1,1,after period ended,1,knows modern method,used modern method,knows sourc.mod.meth,no,15,15,5,wants after 2+ years,secondary,reads easily,"sales, services",no,no,junior high,yes,senior high,modern method,1,1,1,1,0,0,0,0,0
11128,74500 72 2,32,secondary,reads easily,no,yes,no,no,no,catholic,2,3,middle of the cycle,3,knows modern method,used only trad. meth,knows sourc.mod.meth,no,24,24,5,undecided,higher,reads easily,"prof, tech, cleric.",no,no,junior high,no,university,traditional method,0,1,1,1,0,0,0,0,1
11129,74500 75 2,44,primary,reads easily,no,no,yes,no,yes,muslim,1,8,after period ended,5,knows modern method,used modern method,knows sourc.mod.meth,no,19,19,3,wants no more,primary,reads easily,"sales, services",yes,yes,primary school,yes,primary,modern method,1,1,1,1,0,0,1,1,0
11130,74500 82 2,25,secondary,reads easily,yes,yes,no,no,no,muslim,3,4,middle of the cycle,4,knows modern method,never used,knows sourc.mod.meth,no,17,17,7,wants no more,secondary,reads easily,"sales, services",no,no,junior high,yes,senior high,no method,1,1,1,1,0,0,0,0,0


In [24]:
#now check our updated vars to see what we got for each column
for column in non_preg.loc[:, 'Religion_dup':].columns:
    print(num_of_vars(non_preg, column))

Num of Religion_dup variables: 2
1    9330
0    1802
Name: Religion_dup, dtype: int64 

None
Num of Num_children_in_residence_dup variables: 2
1    6605
0    4527
Name: Num_children_in_residence_dup, dtype: int64 

None
Num of Knowledge_of_ovulatory_cycle_dup variables: 2
1    6766
0    4366
Name: Knowledge_of_ovulatory_cycle_dup, dtype: int64 

None
Num of Knows_method_source_dup variables: 2
1    10358
0      774
Name: Knows_method_source_dup, dtype: int64 

None
Num of currently_abstaining_dup variables: 2
0    10483
1      649
Name: currently_abstaining_dup, dtype: int64 

None
Num of desire_for_more_kids_dup variables: 2
0    10123
1     1009
Name: desire_for_more_kids_dup, dtype: int64 

None
Num of Woman_worked_after_marriage_dup variables: 2
1    6133
0    4999
Name: Woman_worked_after_marriage_dup, dtype: int64 

None
Num of Woman_currently_working_dup variables: 2
0    5681
1    5451
Name: Woman_currently_working_dup, dtype: int64 

None
Num of heard_FP_message_in_last_month_

## Grouping changes

In [25]:
has_transp = []
for i in range(len(non_preg)):
    if str(non_preg.loc[i, 'Has_transportation1']) == 'yes' or str(non_preg.loc[i, 'Has_transportation2']) == 'yes':
        has_transp.append(1)
    else:
        has_transp.append(0)


non_preg['Has_transportation'] = has_transp


Change the media columns to binary, then take the sum of them to get media exposure level

In [26]:
make_binary(non_preg, 'Reads_newspaper_once_per_week', 'yes', 1)
make_binary(non_preg, 'Has_radio', 'yes', 1)
make_binary(non_preg, 'Has_TV', 'yes', 1)

,caseid,Age,Education_level,Literacy,Reads_newspaper_once_per_week,Has_radio,Has_TV,Has_transportation1,Has_transportation2,Religion,Num_children_in_residence,Num_children_born,Knowledge_of_ovulatory_cycle,Num_living_children,Knowledge_of_methods,Used_any_method,Knows_method_source,currently_abstaining,age_at_first_marriage,age_at_first_intercourse,freq_of_intercourse,desire_for_more_kids,Husbands_edu_level,Husbands_literacy,Husbands_occupation,Woman_worked_after_marriage,Woman_currently_working,highest_level_of_edu,heard_FP_message_in_last_month,hubby_highest_level_edu,Current_method,Religion_dup,Num_children_in_residence_dup,Knowledge_of_ovulatory_cycle_dup,Knows_method_source_dup,currently_abstaining_dup,desire_for_more_kids_dup,Woman_worked_after_marriage_dup,Woman_currently_working_dup,heard_FP_message_in_last_month_dup,Has_transportation,Reads_newspaper_once_per_week_dup,Has_radio_dup,Has_TV_dup
0,11 1 1 2,41,secondary,reads easily,no,yes,no,yes,no,muslim,0,6,after period ended,5,knows modern method,used modern method,knows sourc.mod.meth,no,15,15,4,wants no more,primary,reads easily,agriculture,yes,yes,junior high,no,primary,no method,1,0,1,1,0,0,1,1,1,1,0,1,0
1,11 1 4 4,27,secondary,reads easily,yes,yes,yes,yes,no,muslim,4,5,after period ended,5,knows modern method,never used,knows sourc.mod.meth,yes,18,18,no intercourse,"wants, unsure timing",secondary,reads easily,"prof, tech, cleric.",no,no,senior high,no,senior high,no method,1,1,1,1,1,0,0,0,1,1,1,1,1
2,11 1 6 1,43,secondary,reads easily,yes,no,no,no,no,muslim,0,8,middle of the cycle,8,knows modern method,used modern method,knows sourc.mod.meth,no,15,15,NaN,NaN,secondary,reads easily,"sales, services",no,no,junior high,no,junior high,no method,1,0,1,1,0,0,0,0,1,0,1,0,0
3,11 1 8 2,41,primary,reads easily,no,yes,no,yes,no,muslim,2,9,after period ended,9,knows modern method,never used,knows sourc.mod.meth,no,18,18,3,"wants, unsure timing",secondary,reads easily,"prof, tech, cleric.",yes,yes,primary school,no,senior high,no method,1,1,1,1,0,0,1,1,1,1,0,1,0
4,11 1 9 2,29,secondary,reads easily,yes,yes,no,yes,no,muslim,1,3,after period ended,2,knows modern method,never used,knows sourc.mod.meth,no,17,17,5,"wants, unsure timing",secondary,reads easily,agriculture,no,no,junior high,no,senior high,no method,1,1,1,1,0,0,0,0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11127,74500 66 2,17,secondary,reads easily,yes,no,no,no,no,muslim,1,1,after period ended,1,knows modern method,used modern method,knows sourc.mod.meth,no,15,15,5,wants after 2+ years,secondary,reads easily,"sales, services",no,no,junior high,yes,senior high,modern method,1,1,1,1,0,0,0,0,0,0,1,0,0
11128,74500 72 2,32,secondary,reads easily,no,yes,no,no,no,catholic,2,3,middle of the cycle,3,knows modern method,used only trad. meth,knows sourc.mod.meth,no,24,24,5,undecided,higher,reads easily,"prof, tech, cleric.",no,no,junior high,no,university,traditional method,0,1,1,1,0,0,0,0,1,0,0,1,0
11129,74500 75 2,44,primary,reads easily,no,no,yes,no,yes,muslim,1,8,after period ended,5,knows modern method,used modern method,knows sourc.mod.meth,no,19,19,3,wants no more,primary,reads easily,"sales, services",yes,yes,primary school,yes,primary,modern method,1,1,1,1,0,0,1,1,0,1,0,0,1
11130,74500 82 2,25,secondary,reads easily,yes,yes,no,no,no,muslim,3,4,middle of the cycle,4,knows modern method,never used,knows sourc.mod.meth,no,17,17,7,wants no more,secondary,reads easily,"sales, services",no,no,junior high,yes,senior high,no method,1,1,1,1,0,0,0,0,0,0,1,1,0


In [27]:
good_media = []

for i in range(len(non_preg)):
    tot = non_preg.loc[i, 'Has_radio_dup'] + non_preg.loc[i, 'Has_TV_dup'] + non_preg.loc[i, 'Reads_newspaper_once_per_week_dup']
    good_media.append(tot)
      
        
non_preg['Good_media_exposure'] = good_media

In [28]:
non_preg.head(3)

,caseid,Age,Education_level,Literacy,Reads_newspaper_once_per_week,Has_radio,Has_TV,Has_transportation1,Has_transportation2,Religion,Num_children_in_residence,Num_children_born,Knowledge_of_ovulatory_cycle,Num_living_children,Knowledge_of_methods,Used_any_method,Knows_method_source,currently_abstaining,age_at_first_marriage,age_at_first_intercourse,freq_of_intercourse,desire_for_more_kids,Husbands_edu_level,Husbands_literacy,Husbands_occupation,Woman_worked_after_marriage,Woman_currently_working,highest_level_of_edu,heard_FP_message_in_last_month,hubby_highest_level_edu,Current_method,Religion_dup,Num_children_in_residence_dup,Knowledge_of_ovulatory_cycle_dup,Knows_method_source_dup,currently_abstaining_dup,desire_for_more_kids_dup,Woman_worked_after_marriage_dup,Woman_currently_working_dup,heard_FP_message_in_last_month_dup,Has_transportation,Reads_newspaper_once_per_week_dup,Has_radio_dup,Has_TV_dup,Good_media_exposure
0,11 1 1 2,41,secondary,reads easily,no,yes,no,yes,no,muslim,0,6,after period ended,5,knows modern method,used modern method,knows sourc.mod.meth,no,15,15,4,wants no more,primary,reads easily,agriculture,yes,yes,junior high,no,primary,no method,1,0,1,1,0,0,1,1,1,1,0,1,0,1
1,11 1 4 4,27,secondary,reads easily,yes,yes,yes,yes,no,muslim,4,5,after period ended,5,knows modern method,never used,knows sourc.mod.meth,yes,18,18,no intercourse,"wants, unsure timing",secondary,reads easily,"prof, tech, cleric.",no,no,senior high,no,senior high,no method,1,1,1,1,1,0,0,0,1,1,1,1,1,3
2,11 1 6 1,43,secondary,reads easily,yes,no,no,no,no,muslim,0,8,middle of the cycle,8,knows modern method,used modern method,knows sourc.mod.meth,no,15,15,NaN,NaN,secondary,reads easily,"sales, services",no,no,junior high,no,junior high,no method,1,0,1,1,0,0,0,0,1,0,1,0,0,1


## Make the value changes next

age_at_first_intercourse: replace 'inconsistent' and 'refused to answer' with the median

frequency_of_intercourse: replace 'no intercourse' with 0 and 'refused to answer' with the median

In [29]:
import numpy as np

In [30]:
#replace the non_numericals with nans for age at first intercourse
non_preg['age_at_first_intercourse'] = non_preg['age_at_first_intercourse'].replace('inconsistent', np.nan).replace('not had intercourse', np.nan).replace('refused to answer', np.nan)


In [31]:
#find median
non_preg['age_at_first_intercourse'].astype(float).median(axis = 0, skipna = True) 

17.0

In [32]:
#fill with the median
non_preg['age_at_first_intercourse'].fillna(17.0, inplace = True)

now fix the freq column

In [33]:
#replace the non_numericals with nans for age at frequency of intercourse
non_preg['freq_of_intercourse'] = non_preg['freq_of_intercourse'].replace('no intercourse', np.nan).replace('refused to answer', np.nan)


In [34]:
non_preg['freq_of_intercourse'].astype(float).median(axis = 0, skipna = True) 

4.0

In [35]:
#fill with the median
non_preg['freq_of_intercourse'].fillna(4.0, inplace = True)

In [36]:
#find median
non_preg.median(axis = 0) 

Age                                   32.0
Num_children_in_residence              1.0
Num_children_born                      3.0
Num_living_children                    3.0
age_at_first_marriage                 17.0
age_at_first_intercourse              17.0
freq_of_intercourse                    4.0
Religion_dup                           1.0
Num_children_in_residence_dup          1.0
Knowledge_of_ovulatory_cycle_dup       1.0
Knows_method_source_dup                1.0
currently_abstaining_dup               0.0
desire_for_more_kids_dup               0.0
Woman_worked_after_marriage_dup        1.0
Woman_currently_working_dup            0.0
heard_FP_message_in_last_month_dup     1.0
Has_transportation                     1.0
Reads_newspaper_once_per_week_dup      0.0
Has_radio_dup                          1.0
Has_TV_dup                             0.0
Good_media_exposure                    1.0
dtype: float64

## next start changing the ordinal values.
### Won't change them all rn, but i will change the ones used in my FSM, then come back later and change the rest if i have more time

In [37]:
#drop the nan rows since last rows since didn't even have a caseid
non_preg.dropna(inplace = True)

In [38]:
#reset the index since we need it to be the same when we use enumerate later
non_preg.reset_index(inplace = True, drop = True)

In [39]:
non_preg.tail(3)

,caseid,Age,Education_level,Literacy,Reads_newspaper_once_per_week,Has_radio,Has_TV,Has_transportation1,Has_transportation2,Religion,Num_children_in_residence,Num_children_born,Knowledge_of_ovulatory_cycle,Num_living_children,Knowledge_of_methods,Used_any_method,Knows_method_source,currently_abstaining,age_at_first_marriage,age_at_first_intercourse,freq_of_intercourse,desire_for_more_kids,Husbands_edu_level,Husbands_literacy,Husbands_occupation,Woman_worked_after_marriage,Woman_currently_working,highest_level_of_edu,heard_FP_message_in_last_month,hubby_highest_level_edu,Current_method,Religion_dup,Num_children_in_residence_dup,Knowledge_of_ovulatory_cycle_dup,Knows_method_source_dup,currently_abstaining_dup,desire_for_more_kids_dup,Woman_worked_after_marriage_dup,Woman_currently_working_dup,heard_FP_message_in_last_month_dup,Has_transportation,Reads_newspaper_once_per_week_dup,Has_radio_dup,Has_TV_dup,Good_media_exposure
7277,74500 75 2,44,primary,reads easily,no,no,yes,no,yes,muslim,1,8,after period ended,5,knows modern method,used modern method,knows sourc.mod.meth,no,19,19.0,3.0,wants no more,primary,reads easily,"sales, services",yes,yes,primary school,yes,primary,modern method,1,1,1,1,0,0,1,1,0,1,0,0,1,1
7278,74500 82 2,25,secondary,reads easily,yes,yes,no,no,no,muslim,3,4,middle of the cycle,4,knows modern method,never used,knows sourc.mod.meth,no,17,17.0,7.0,wants no more,secondary,reads easily,"sales, services",no,no,junior high,yes,senior high,no method,1,1,1,1,0,0,0,0,0,0,1,1,0,2
7279,74500 85 2,22,secondary,reads easily,yes,no,no,no,no,muslim,1,1,after period ended,1,knows modern method,never used,knows sourc.mod.meth,no,21,21.0,2.0,wants after 2+ years,higher,reads easily,"prof, tech, cleric.",yes,yes,senior high,no radio or tv,academy,no method,1,1,1,1,0,0,1,1,1,0,1,0,0,1


In [40]:
non_preg.shape

(7280, 45)

In [41]:
#Woohoo! Okay, now down to business

In [42]:
#check what we have for hubby's 
non_preg.Husbands_edu_level.value_counts()

primary         3721
secondary       2680
higher           570
no education     288
don't know        21
Name: Husbands_edu_level, dtype: int64

In [43]:
#create all the maps to change the vars
education_map = {'no education': 0, 'primary': 1, 'secondary': 2, "don't know": 2,'higher':3}
literacy_map = {'cannot read': 0, 'reads with difficult': 1, 'reads easily': 2}
know_methods_map = {'knows no method':0, 'knows modern method': 1, 'knows only trad. mth': 2}
used_methods_map = {'never used': 0, 'used only trad. meth': 1, 'used modern method': 2}
knows_source_map = {"doesn't know source": 0, 'knows sourc.mod.meth':1}
current_method_map = {'no method': 0, 'traditional method':1, 'modern method': 2}

In [44]:
change_vars(education_map, non_preg, 'Education_level')
change_vars(education_map, non_preg, 'Husbands_edu_level')
change_vars(literacy_map, non_preg, 'Literacy')
change_vars(know_methods_map, non_preg, 'Knowledge_of_methods')
change_vars(used_methods_map, non_preg, 'Used_any_method')
change_vars(knows_source_map, non_preg, 'Knows_method_source')
change_vars(current_method_map, non_preg, 'Current_method')

,caseid,Age,Education_level,Literacy,Reads_newspaper_once_per_week,Has_radio,Has_TV,Has_transportation1,Has_transportation2,Religion,Num_children_in_residence,Num_children_born,Knowledge_of_ovulatory_cycle,Num_living_children,Knowledge_of_methods,Used_any_method,Knows_method_source,currently_abstaining,age_at_first_marriage,age_at_first_intercourse,freq_of_intercourse,desire_for_more_kids,Husbands_edu_level,Husbands_literacy,Husbands_occupation,Woman_worked_after_marriage,Woman_currently_working,highest_level_of_edu,heard_FP_message_in_last_month,hubby_highest_level_edu,Current_method,Religion_dup,Num_children_in_residence_dup,Knowledge_of_ovulatory_cycle_dup,Knows_method_source_dup,currently_abstaining_dup,desire_for_more_kids_dup,Woman_worked_after_marriage_dup,Woman_currently_working_dup,heard_FP_message_in_last_month_dup,Has_transportation,Reads_newspaper_once_per_week_dup,Has_radio_dup,Has_TV_dup,Good_media_exposure,Education_level_dup,Husbands_edu_level_dup,Literacy_dup,Knowledge_of_methods_dup,Used_any_method_dup,Current_method_dup
0,11 1 1 2,41,secondary,reads easily,no,yes,no,yes,no,muslim,0,6,after period ended,5,knows modern method,used modern method,knows sourc.mod.meth,no,15,15.0,4.0,wants no more,primary,reads easily,agriculture,yes,yes,junior high,no,primary,no method,1,0,1,1,0,0,1,1,1,1,0,1,0,1,2,1,2,1,2,0
1,11 1 4 4,27,secondary,reads easily,yes,yes,yes,yes,no,muslim,4,5,after period ended,5,knows modern method,never used,knows sourc.mod.meth,yes,18,18.0,4.0,"wants, unsure timing",secondary,reads easily,"prof, tech, cleric.",no,no,senior high,no,senior high,no method,1,1,1,1,1,0,0,0,1,1,1,1,1,3,2,2,2,1,0,0
2,11 1 8 2,41,primary,reads easily,no,yes,no,yes,no,muslim,2,9,after period ended,9,knows modern method,never used,knows sourc.mod.meth,no,18,18.0,3.0,"wants, unsure timing",secondary,reads easily,"prof, tech, cleric.",yes,yes,primary school,no,senior high,no method,1,1,1,1,0,0,1,1,1,1,0,1,0,1,1,2,2,1,0,0
3,11 1 9 2,29,secondary,reads easily,yes,yes,no,yes,no,muslim,1,3,after period ended,2,knows modern method,never used,knows sourc.mod.meth,no,17,17.0,5.0,"wants, unsure timing",secondary,reads easily,agriculture,no,no,junior high,no,senior high,no method,1,1,1,1,0,0,0,0,1,1,1,1,0,2,2,2,2,1,0,0
4,11 1 12 3,25,higher,reads easily,yes,yes,yes,yes,yes,muslim,2,2,after period ended,2,knows modern method,used only trad. meth,knows sourc.mod.meth,no,19,19.0,2.0,"wants, unsure timing",secondary,reads easily,"prof, tech, cleric.",no,no,academy,yes,senior high,traditional method,1,1,1,1,0,0,0,0,0,1,1,1,1,3,3,2,2,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7275,74500 66 2,17,secondary,reads easily,yes,no,no,no,no,muslim,1,1,after period ended,1,knows modern method,used modern method,knows sourc.mod.meth,no,15,15.0,5.0,wants after 2+ years,secondary,reads easily,"sales, services",no,no,junior high,yes,senior high,modern method,1,1,1,1,0,0,0,0,0,0,1,0,0,1,2,2,2,1,2,2
7276,74500 72 2,32,secondary,reads easily,no,yes,no,no,no,catholic,2,3,middle of the cycle,3,knows modern method,used only trad. meth,knows sourc.mod.meth,no,24,24.0,5.0,undecided,higher,reads easily,"prof, tech, cleric.",no,no,junior high,no,university,traditional method,0,1,1,1,0,0,0,0,1,0,0,1,0,1,2,3,2,1,1,1
7277,74500 75 2,44,primary,reads easily,no,no,yes,no,yes,muslim,1,8,after period ended,5,knows modern method,used modern method,knows sourc.mod.meth,no,19,19.0,3.0,wants no more,primary,reads easily,"sales, services",yes,yes,primary school,yes,primary,modern method,1,1,1,1,0,0,1,1,0,1,0,0,1,1,1,1,2,1,2,2
7278,74500 82 2,25,secondary,reads easily,yes,yes,no,no,no,muslim,3,4,middle of the cycle,4,knows modern method,never used,knows sourc.mod.meth,no,17,17.0,7.0,wants no more,secondary,reads easily,"sales, services",no,no,junior high,yes,senior high,no method,1,1,1,1,0,0,0,0,0,0,1,1,0,

In [45]:
change_vars(literacy_map, non_preg, 'Husbands_literacy')

,caseid,Age,Education_level,Literacy,Reads_newspaper_once_per_week,Has_radio,Has_TV,Has_transportation1,Has_transportation2,Religion,Num_children_in_residence,Num_children_born,Knowledge_of_ovulatory_cycle,Num_living_children,Knowledge_of_methods,Used_any_method,Knows_method_source,currently_abstaining,age_at_first_marriage,age_at_first_intercourse,freq_of_intercourse,desire_for_more_kids,Husbands_edu_level,Husbands_literacy,Husbands_occupation,Woman_worked_after_marriage,Woman_currently_working,highest_level_of_edu,heard_FP_message_in_last_month,hubby_highest_level_edu,Current_method,Religion_dup,Num_children_in_residence_dup,Knowledge_of_ovulatory_cycle_dup,Knows_method_source_dup,currently_abstaining_dup,desire_for_more_kids_dup,Woman_worked_after_marriage_dup,Woman_currently_working_dup,heard_FP_message_in_last_month_dup,Has_transportation,Reads_newspaper_once_per_week_dup,Has_radio_dup,Has_TV_dup,Good_media_exposure,Education_level_dup,Husbands_edu_level_dup,Literacy_dup,Knowledge_of_methods_dup,Used_any_method_dup,Current_method_dup,Husbands_literacy_dup
0,11 1 1 2,41,secondary,reads easily,no,yes,no,yes,no,muslim,0,6,after period ended,5,knows modern method,used modern method,knows sourc.mod.meth,no,15,15.0,4.0,wants no more,primary,reads easily,agriculture,yes,yes,junior high,no,primary,no method,1,0,1,1,0,0,1,1,1,1,0,1,0,1,2,1,2,1,2,0,2
1,11 1 4 4,27,secondary,reads easily,yes,yes,yes,yes,no,muslim,4,5,after period ended,5,knows modern method,never used,knows sourc.mod.meth,yes,18,18.0,4.0,"wants, unsure timing",secondary,reads easily,"prof, tech, cleric.",no,no,senior high,no,senior high,no method,1,1,1,1,1,0,0,0,1,1,1,1,1,3,2,2,2,1,0,0,2
2,11 1 8 2,41,primary,reads easily,no,yes,no,yes,no,muslim,2,9,after period ended,9,knows modern method,never used,knows sourc.mod.meth,no,18,18.0,3.0,"wants, unsure timing",secondary,reads easily,"prof, tech, cleric.",yes,yes,primary school,no,senior high,no method,1,1,1,1,0,0,1,1,1,1,0,1,0,1,1,2,2,1,0,0,2
3,11 1 9 2,29,secondary,reads easily,yes,yes,no,yes,no,muslim,1,3,after period ended,2,knows modern method,never used,knows sourc.mod.meth,no,17,17.0,5.0,"wants, unsure timing",secondary,reads easily,agriculture,no,no,junior high,no,senior high,no method,1,1,1,1,0,0,0,0,1,1,1,1,0,2,2,2,2,1,0,0,2
4,11 1 12 3,25,higher,reads easily,yes,yes,yes,yes,yes,muslim,2,2,after period ended,2,knows modern method,used only trad. meth,knows sourc.mod.meth,no,19,19.0,2.0,"wants, unsure timing",secondary,reads easily,"prof, tech, cleric.",no,no,academy,yes,senior high,traditional method,1,1,1,1,0,0,0,0,0,1,1,1,1,3,3,2,2,1,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7275,74500 66 2,17,secondary,reads easily,yes,no,no,no,no,muslim,1,1,after period ended,1,knows modern method,used modern method,knows sourc.mod.meth,no,15,15.0,5.0,wants after 2+ years,secondary,reads easily,"sales, services",no,no,junior high,yes,senior high,modern method,1,1,1,1,0,0,0,0,0,0,1,0,0,1,2,2,2,1,2,2,2
7276,74500 72 2,32,secondary,reads easily,no,yes,no,no,no,catholic,2,3,middle of the cycle,3,knows modern method,used only trad. meth,knows sourc.mod.meth,no,24,24.0,5.0,undecided,higher,reads easily,"prof, tech, cleric.",no,no,junior high,no,university,traditional method,0,1,1,1,0,0,0,0,1,0,0,1,0,1,2,3,2,1,1,1,2
7277,74500 75 2,44,primary,reads easily,no,no,yes,no,yes,muslim,1,8,after period ended,5,knows modern method,used modern method,knows sourc.mod.meth,no,19,19.0,3.0,wants no more,primary,reads easily,"sales, services",yes,yes,primary school,yes,primary,modern method,1,1,1,1,0,0,1,1,0,1,0,0,1,1,1,1,2,1,2,2,2
7278,74500 82 2,25,secondary,reads easily,yes,yes,no,no,no,muslim,3,4,middle of the cycle,4,knows modern method,never used,knows sourc.mod.meth,no,17,17.0,7.0,wants no more,secondary,reads easily,"sales, services",no,no,junior high,yes,senior

In [46]:
non_preg.head(3)

,caseid,Age,Education_level,Literacy,Reads_newspaper_once_per_week,Has_radio,Has_TV,Has_transportation1,Has_transportation2,Religion,Num_children_in_residence,Num_children_born,Knowledge_of_ovulatory_cycle,Num_living_children,Knowledge_of_methods,Used_any_method,Knows_method_source,currently_abstaining,age_at_first_marriage,age_at_first_intercourse,freq_of_intercourse,desire_for_more_kids,Husbands_edu_level,Husbands_literacy,Husbands_occupation,Woman_worked_after_marriage,Woman_currently_working,highest_level_of_edu,heard_FP_message_in_last_month,hubby_highest_level_edu,Current_method,Religion_dup,Num_children_in_residence_dup,Knowledge_of_ovulatory_cycle_dup,Knows_method_source_dup,currently_abstaining_dup,desire_for_more_kids_dup,Woman_worked_after_marriage_dup,Woman_currently_working_dup,heard_FP_message_in_last_month_dup,Has_transportation,Reads_newspaper_once_per_week_dup,Has_radio_dup,Has_TV_dup,Good_media_exposure,Education_level_dup,Husbands_edu_level_dup,Literacy_dup,Knowledge_of_methods_dup,Used_any_method_dup,Current_method_dup,Husbands_literacy_dup
0,11 1 1 2,41,secondary,reads easily,no,yes,no,yes,no,muslim,0,6,after period ended,5,knows modern method,used modern method,knows sourc.mod.meth,no,15,15.0,4.0,wants no more,primary,reads easily,agriculture,yes,yes,junior high,no,primary,no method,1,0,1,1,0,0,1,1,1,1,0,1,0,1,2,1,2,1,2,0,2
1,11 1 4 4,27,secondary,reads easily,yes,yes,yes,yes,no,muslim,4,5,after period ended,5,knows modern method,never used,knows sourc.mod.meth,yes,18,18.0,4.0,"wants, unsure timing",secondary,reads easily,"prof, tech, cleric.",no,no,senior high,no,senior high,no method,1,1,1,1,1,0,0,0,1,1,1,1,1,3,2,2,2,1,0,0,2
2,11 1 8 2,41,primary,reads easily,no,yes,no,yes,no,muslim,2,9,after period ended,9,knows modern method,never used,knows sourc.mod.meth,no,18,18.0,3.0,"wants, unsure timing",secondary,reads easily,"prof, tech, cleric.",yes,yes,primary school,no,senior high,no method,1,1,1,1,0,0,1,1,1,1,0,1,0,1,1,2,2,1,0,0,2


In [47]:
non_preg.tail(3)

,caseid,Age,Education_level,Literacy,Reads_newspaper_once_per_week,Has_radio,Has_TV,Has_transportation1,Has_transportation2,Religion,Num_children_in_residence,Num_children_born,Knowledge_of_ovulatory_cycle,Num_living_children,Knowledge_of_methods,Used_any_method,Knows_method_source,currently_abstaining,age_at_first_marriage,age_at_first_intercourse,freq_of_intercourse,desire_for_more_kids,Husbands_edu_level,Husbands_literacy,Husbands_occupation,Woman_worked_after_marriage,Woman_currently_working,highest_level_of_edu,heard_FP_message_in_last_month,hubby_highest_level_edu,Current_method,Religion_dup,Num_children_in_residence_dup,Knowledge_of_ovulatory_cycle_dup,Knows_method_source_dup,currently_abstaining_dup,desire_for_more_kids_dup,Woman_worked_after_marriage_dup,Woman_currently_working_dup,heard_FP_message_in_last_month_dup,Has_transportation,Reads_newspaper_once_per_week_dup,Has_radio_dup,Has_TV_dup,Good_media_exposure,Education_level_dup,Husbands_edu_level_dup,Literacy_dup,Knowledge_of_methods_dup,Used_any_method_dup,Current_method_dup,Husbands_literacy_dup
7277,74500 75 2,44,primary,reads easily,no,no,yes,no,yes,muslim,1,8,after period ended,5,knows modern method,used modern method,knows sourc.mod.meth,no,19,19.0,3.0,wants no more,primary,reads easily,"sales, services",yes,yes,primary school,yes,primary,modern method,1,1,1,1,0,0,1,1,0,1,0,0,1,1,1,1,2,1,2,2,2
7278,74500 82 2,25,secondary,reads easily,yes,yes,no,no,no,muslim,3,4,middle of the cycle,4,knows modern method,never used,knows sourc.mod.meth,no,17,17.0,7.0,wants no more,secondary,reads easily,"sales, services",no,no,junior high,yes,senior high,no method,1,1,1,1,0,0,0,0,0,0,1,1,0,2,2,2,2,1,0,0,2
7279,74500 85 2,22,secondary,reads easily,yes,no,no,no,no,muslim,1,1,after period ended,1,knows modern method,never used,knows sourc.mod.meth,no,21,21.0,2.0,wants after 2+ years,higher,reads easily,"prof, tech, cleric.",yes,yes,senior high,no radio or tv,academy,no method,1,1,1,1,0,0,1,1,1,0,1,0,0,1,2,3,2,1,0,0,2


In [48]:
non_preg.shape

(7280, 52)

## Make a new df with just the transformed columns now for our second iteration

In [49]:
non_preg2 = non_preg[['caseid', 'Age', 'Num_children_born',
                      'Num_living_children', 'age_at_first_marriage',
                      'age_at_first_intercourse', 
                      'freq_of_intercourse',
                      'Religion_dup', 'Num_children_in_residence_dup',
                      'Knowledge_of_ovulatory_cycle_dup', 
                      'Knows_method_source_dup', 
                      'currently_abstaining_dup', 
                      'desire_for_more_kids_dup', 
                      'Woman_worked_after_marriage_dup', 
                      'Woman_currently_working_dup', 
                      'heard_FP_message_in_last_month_dup', 
                      'Has_transportation', 
                      'Good_media_exposure', 'Education_level_dup', 
                      'Literacy_dup', 'Husbands_edu_level_dup', 
                      'Husbands_literacy_dup',
                      'Knowledge_of_methods_dup', 'Used_any_method_dup', 
                      'Current_method_dup']]


In [50]:
non_preg2.head(3)

,caseid,Age,Num_children_born,Num_living_children,age_at_first_marriage,age_at_first_intercourse,freq_of_intercourse,Religion_dup,Num_children_in_residence_dup,Knowledge_of_ovulatory_cycle_dup,Knows_method_source_dup,currently_abstaining_dup,desire_for_more_kids_dup,Woman_worked_after_marriage_dup,Woman_currently_working_dup,heard_FP_message_in_last_month_dup,Has_transportation,Good_media_exposure,Education_level_dup,Literacy_dup,Husbands_edu_level_dup,Husbands_literacy_dup,Knowledge_of_methods_dup,Used_any_method_dup,Current_method_dup
0,11 1 1 2,41,6,5,15,15.0,4.0,1,0,1,1,0,0,1,1,1,1,1,2,2,1,2,1,2,0
1,11 1 4 4,27,5,5,18,18.0,4.0,1,1,1,1,1,0,0,0,1,1,3,2,2,2,2,1,0,0
2,11 1 8 2,41,9,9,18,18.0,3.0,1,1,1,1,0,0,1,1,1,1,1,1,2,2,2,1,0,0


In [51]:
non_preg2 = non_preg2.rename(columns = {'Num_children_in_residence_dup':'children_in_residence', 'Good_media_exposure':'media_exposure_level'})


In [52]:
non_preg2.head(3)

,caseid,Age,Num_children_born,Num_living_children,age_at_first_marriage,age_at_first_intercourse,freq_of_intercourse,Religion_dup,children_in_residence,Knowledge_of_ovulatory_cycle_dup,Knows_method_source_dup,currently_abstaining_dup,desire_for_more_kids_dup,Woman_worked_after_marriage_dup,Woman_currently_working_dup,heard_FP_message_in_last_month_dup,Has_transportation,media_exposure_level,Education_level_dup,Literacy_dup,Husbands_edu_level_dup,Husbands_literacy_dup,Knowledge_of_methods_dup,Used_any_method_dup,Current_method_dup
0,11 1 1 2,41,6,5,15,15.0,4.0,1,0,1,1,0,0,1,1,1,1,1,2,2,1,2,1,2,0
1,11 1 4 4,27,5,5,18,18.0,4.0,1,1,1,1,1,0,0,0,1,1,3,2,2,2,2,1,0,0
2,11 1 8 2,41,9,9,18,18.0,3.0,1,1,1,1,0,0,1,1,1,1,1,1,2,2,2,1,0,0


In [53]:
non_preg2.to_pickle('second_model_df.pkl')